## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [2]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [3]:
df = pd.read_csv('ab_data.csv')
df.head(5)

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


b. Use the below cell to find the number of rows in the dataset.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
user_id         294478 non-null int64
timestamp       294478 non-null object
group           294478 non-null object
landing_page    294478 non-null object
converted       294478 non-null int64
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


c. The number of unique users in the dataset.

In [5]:
df['user_id'].nunique()

290584

d. The proportion of users converted.

In [6]:
df['converted'].mean()

0.11965919355605512

e. The number of times the `new_page` and `treatment` don't line up.

In [7]:
line_up1 = df.query('landing_page != "new_page" and group == "treatment"')['user_id'].count()
line_up2 = df.query('landing_page == "new_page" and group != "treatment"')['user_id'].count()
line_up = line_up1 + line_up2
print(line_up)

3893


f. Do any of the rows have missing values?

No.

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In [56]:
df_exclude1 = df.query('landing_page == "old_page" and group == "treatment"')
df_exclude2 = df.query('landing_page == "new_page" and group == "control"')
mis_match = pd.concat([df_exclude1, df_exclude2])
df2 = df
df2 = df2.drop(mis_match.index)
df2.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [9]:
# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [10]:
a = df2['user_id'].nunique()
b = df2['user_id'].count()
b - a

1

b. There is one **user_id** repeated in **df2**.  What is it?

In [11]:
df2[df2.duplicated('user_id')]

,user_id,timestamp,group,landing_page,converted
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


c. What is the row information for the repeat **user_id**? 

See the result above.

d. Remove **one** of the rows with a duplicate **user_id**, but keep your dataframe as **df2**.

In [12]:
df2[df2['user_id'] == 773192]

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [13]:
df2.drop(labels=1899, axis=0, inplace=True)

`4.` Use **df2** in the below cells to answer the quiz questions related to **Quiz 4** in the classroom.

a. What is the probability of an individual converting regardless of the page they receive?

In [14]:
df2['converted'].mean()

0.11959708724499628

b. Given that an individual was in the `control` group, what is the probability they converted?

In [15]:
df2[df2['group'] == 'control']['converted'].mean()

0.1203863045004612

c. Given that an individual was in the `treatment` group, what is the probability they converted?

In [16]:
df2[df2['group'] == 'treatment']['converted'].mean()

0.11880806551510564

d. What is the probability that an individual received the new page?

In [17]:
df2[df2['landing_page'] == 'new_page'].count() / len(df2)

user_id         0.500062
timestamp       0.500062
group           0.500062
landing_page    0.500062
converted       0.500062
dtype: float64

e. Use the results in the previous two portions of this question to suggest if you think there is evidence that one page leads to more conversions?  Write your response below.

By observing the probabilities calculated above, I found that 12.04% of the people who received old pages originally converted to new page, while 11.88% of people who receive new page coverted to old page. Therefore, there's only 0.16% difference. Furthermore, 

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

null hypothese: **$p_{old}$** - **$p_{new}$** >= 0

alternative hypothese: **$p_{old}$** - **$p_{new}$** < 0

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

a. What is the **convert rate** for $p_{new}$ under the null? 

In [18]:
p_new = df2[df2['landing_page'] == 'new_page']['converted'].mean()
print('p_new',p_new)

p_new 0.118808065515


In [19]:
p_old = df2[df2['landing_page'] == 'old_page']['converted'].mean()
print('p_old',p_old)

p_old 0.1203863045


In [20]:
p_mean = (p_new + p_old) /2
p_mean

0.11959718500778342

Therefore,  $p_{new}$ = $p_{old}$ = $p_{mean}$ = 0.11959718500778342.

b. What is the **convert rate** for $p_{old}$ under the null? <br><br>

In [21]:
diff = p_new - p_old
diff

-0.0015782389853555567

c. What is $n_{new}$?

In [22]:
n_new, n_old = df2['landing_page'].value_counts()
print('n_new', n_new)

n_new 145310


In [23]:
p_mean = (p_new + p_old) / 2
print(p_mean)

0.119597185008


d. What is $n_{old}$?

In [24]:
print('n_old', n_old)

n_old 145274


e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [25]:
new_page_converted = df2[df2['landing_page'] == 'new_page']['converted']

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [26]:
old_page_converted = df2[df2['landing_page'] == 'old_page']['converted']

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [27]:
new_page_sim = np.random.choice([0,1], size=n_new, p = [p_mean, (1 - p_mean)])
old_page_sim = np.random.choice([0,1], size=n_old, p = [p_mean, (1 - p_mean)])
new_sim_mean = new_page_sim.mean()
old_sim_mean = old_page_sim.mean()
sim_diff = new_sim_mean - old_sim_mean
sim_diff

0.0013511672953758769

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in **p_diffs**.

In [28]:
sim_diffs = []
for i in range(10000):
    new_page_sim = np.random.choice([0,1], size=n_new, p = [p_mean, (1 - p_mean)])
    old_page_sim = np.random.choice([0,1], size=n_old, p = [p_mean, (1 - p_mean)])
    sim_diff = new_page_sim.mean() - old_page_sim.mean()
    sim_diffs.append(sim_diff)

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [23]:
plt.hist(sim_diffs)
plt.axvline(diff, color='r', linestyle='dashed', linewidth=1, label="Real difference")
plt.axvline(np.array(sim_diffs).mean(), color='g', linewidth=1, label="sim mean")

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [29]:
greater_diff =[i for i in sim_diffs if i > diff] 
percent = len(greater_diff) / len(sim_diffs)
percent

0.903

k. In words, explain what you just computed in part **j.**.  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

The value is called p-value. By using the definition, if p-value is < 0.5, we can conclude that we reject null hypothesis.However, the p-value is 0.9089, which is greater than 0.5. So we failed to reject null hypothesis, which means that the old page is better than the new one.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [30]:
import statsmodels.api as sm

convert_old = len(df2.query('landing_page == "old_page" and converted == 1'))
convert_new = len(df2.query('landing_page == "new_page" and converted == 1'))
print("n_old",n_old)
print("n_old",n_new)
print('convert_old', convert_old)
print('convert_new', convert_new)

/opt/conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


n_old 145274
n_old 145310
convert_old 17489
convert_new 17264


Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [33]:
z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new],alternative='smaller')
# I added "alternative = 'smaller'" is because the alternative hypothesis is p_old < p_new.
p_value, z_score

(0.90505831275902449, 1.3109241984234394)

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

Here, the p-value is 0.190, which is less than 0.05. Therefore, we can conclude that we cannot reject the null hypothesis, which indicates that the old page is better. As the z score is 1.311, which means that the distance between two vertical lines is nearly 1.31 * standard deviation.

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

Logistic Regression

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a colun for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [57]:
df3 = df2
df3['intercept'] = 1
df3[['control','treatment']] = pd.get_dummies(df3['group'])
df3.head()

,user_id,timestamp,group,landing_page,converted,intercept,control,treatment
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,1,1,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,1,1,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,1,0,1
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,1,1,0


c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [58]:
df3['intercept'] = 1
lm = sm.Logit(df3['converted'], df3[['intercept','control']])


d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [59]:
result = lm.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.366118
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290585
Model:                          Logit   Df Residuals:                   290583
Method:                           MLE   Df Model:                            1
Date:                Sun, 09 Sep 2018   Pseudo R-squ.:               8.085e-06
Time:                        07:12:54   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
                                        LLR p-value:                    0.1897
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -2.0038      0.008   -247.147      0.000      -2.020      -1.988
control        0.0150      0.011      1.312      0.190      -0.007       0.037
==============================================================================
"""

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in the **Part II**?<br><br>  **Hint**: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in the **Part II**?

The null and alternative hypotheses associated with your regression model are:

Null Hypothesis: **$p_{old}$** = **$p_{new}$** 

Alternative Hypothesis: **$p_{old}$** != **$p_{new}$** 

In part II, the null and alternative hypothesis are : 

null hypothesis: **$p_{old}$** - **$p_{new}$** >= 0

alternative hypothesis: **$p_{old}$** - **$p_{new}$** < 0

While in this part, the null and hypothesis just discuss whether  **$p_{old}$** and **$p_{new}$** are the same or not.

In fact, the p-values in two parts have some correlation. By using 1 - 0.19 / 2 = 0.905, we can get the roughly same result. Part III is concerned with if the condition had any effect at all, so it is a two-tailed test. You can actually see this in the formula to interchange the p-values because is essentially taking the two-tailed p-value, cutting it in half for only one of the tails, and then calculating the right side of the tail since we are concerned with which one is bigger.

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

Apparently, there're disadvantages to adding additional terms into the regression model. we may add some trivial terms into the model, and that will not have a large impact on the final result.

I think the user's gender, age and nationality might influence whether or not an individual converts. Also, as the time goes by, the design trend differs, we can also take the time stamp into consideration.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy varaibles.** Provide the statistical output as well as a written response to answer this question.

In [36]:
countries_df = pd.read_csv('countries.csv')
countries_df.head()


,user_id,country
0,834778,UK
1,928468,US
2,822059,UK
3,711597,UK
4,710616,UK


In [63]:
df4 = df3
df4 = df4.merge(countries_df, on='user_id')
df_dummy = pd.get_dummies(data=countries_df, columns=['country'])
df4 = df_dummy.merge(df4, on='user_id')
df4.head()

,user_id,country_CA,country_UK,country_US,timestamp,group,landing_page,converted,intercept,control,treatment,country
0,834778,0,1,0,2017-01-14 23:08:43.304998,control,old_page,0,1,1,0,UK
1,928468,0,0,1,2017-01-23 14:44:16.387854,treatment,new_page,0,1,0,1,US
2,822059,0,1,0,2017-01-16 14:04:14.719771,treatment,new_page,1,1,0,1,UK
3,711597,0,1,0,2017-01-22 03:14:24.763511,control,old_page,0,1,1,0,UK
4,710616,0,1,0,2017-01-16 13:14:44.000513,treatment,new_page,0,1,0,1,UK


In [64]:
df4[['new_page','old_page']] = pd.get_dummies(df4['landing_page'])
df4.head()

,user_id,country_CA,country_UK,country_US,timestamp,group,landing_page,converted,intercept,control,treatment,country,new_page,old_page
0,834778,0,1,0,2017-01-14 23:08:43.304998,control,old_page,0,1,1,0,UK,0,1
1,928468,0,0,1,2017-01-23 14:44:16.387854,treatment,new_page,0,1,0,1,US,1,0
2,822059,0,1,0,2017-01-16 14:04:14.719771,treatment,new_page,1,1,0,1,UK,1,0
3,711597,0,1,0,2017-01-22 03:14:24.763511,control,old_page,0,1,1,0,UK,0,1
4,710616,0,1,0,2017-01-16 13:14:44.000513,treatment,new_page,0,1,0,1,UK,1,0


h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [65]:
lm1 = sm.Logit(df4['converted'], df4[['intercept','old_page','country_CA','country_UK']])
result1 = lm1.fit()
result1.summary()

Optimization terminated successfully.
         Current function value: 0.366113
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              converted   No. Observations:               290584
Model:                          Logit   Df Residuals:                   290580
Method:                           MLE   Df Model:                            3
Date:                Sat, 08 Sep 2018   Pseudo R-squ.:               2.323e-05
Time:                        18:49:09   Log-Likelihood:            -1.0639e+05
converged:                       True   LL-Null:                   -1.0639e+05
                                        LLR p-value:                    0.1760
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -2.0042      0.009   -224.560      0.000      -2.022      -1.987
old_page       0.0149      0.011      1.307      0.191      -0.007       0.037
country_CA    -0.0408      0.027     -1.516      0.130      -0.093       0.012
country_UK     0.0099      0.013      0.743      0.457      -0.016       0.036
==============================================================================
"""

In [49]:
coff_CA = np.exp(0.0408)
coff_CA

1.0416437559600236

In [50]:
coff_UK = np.exp(0.0099)
coff_UK

1.0099491671175422

In [60]:
coff_old_page = np.exp(0.0149)
coff_old_page

1.0150115583846535

In [68]:
df4.groupby('country')['new_page'].mean()

country
CA    0.503552
UK    0.498247
US    0.500459
Name: new_page, dtype: float64

## Conclusion:
By observing the probability that an individual received whether new_page or old_page, we can see that the user have nearly 50% to receive either of the two version.
As the logistic regression model above, we noticed that none of the p-values related to countries and pages is less than 0.05. So we can conclude that we cannot reject the null hypothesis, which indicates that there's no improvement for the new pages. The company should re-develop the new website again.

<a id='conclusions'></a>
## Finishing Up

> Congratulations!  You have reached the end of the A/B Test Results project!  This is the final project in Term 1.  You should be very proud of all you have accomplished!

> **Tip**: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.


## Directions to Submit

> Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).

> Alternatively, you can download this report as .html via the **File** > **Download as** submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.

> Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!

In [ ]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])